<a href="https://colab.research.google.com/github/Mamediarra390/GDA_Live_coding_FML23/blob/main/Mame_Diarra_Diop_Live_coding_GDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GDA Implementation.

Implement the Gaussian Discriminant Analysis (GDA) learning algorithm following the steps as discussed in class.

INSTRUCTION: Rename your notebook as: <br>
`firstName_LastName_Live_coding_GDA.ipynb`.

Notes: 
* Do not use any built-in functions to complete a task;
* Do not import additional libraries.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification

In [2]:
# Generate data
def generate_data():
  x, y = make_classification(n_samples= 1000, n_features=3, n_redundant=0, 
                           n_informative=3, random_state=1, 
                           n_clusters_per_class=1)
  
  return x,y

x,y= generate_data()
print(x.shape, y.shape)

(1000, 3) (1000,)


In [3]:
def split_data(x,y, train_size= 0.8):
    # shuffle the data to randomize the train/test split
    indice = np.arange(x.shape[0])
    np.random.permutation(indice)
    x= x[indice]
    y = y[indice]
    num = int(train_size * x.shape[0])
    x_train = x[:num]
    y_train = y[:num]
    x_test = x[num:]
    y_test = y[num:]
    return x_train, x_test, y_train, y_test


In [4]:
x_train, x_test, y_train, y_test= split_data(x,y, train_size= 0.8) # split your data into x_train, x_test, y_train, y_test
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(800, 3) (800,) (200, 3) (200,)


In [5]:
def covariance(x, mu):

  # Easy way: cov= np.cov(x, rowvar=0) but do not use it. One can use it to assess his/her result.
  N,d = x.shape
  sigma = np.zeros((d,d))
  K=mu.shape[0]
  for k in range(K):
    for i in range(d):
      for j in range(d):
        sigma[i,j] = 1/(N-1)*np.sum((x[:,i] - mu[i]) * (x[:,j] - mu[j])) 
  return sigma

In [6]:
covariance(x, x.mean(axis=0))

array([[1.84495325, 0.02790646, 1.00137533],
       [0.02790646, 1.00170721, 0.05539176],
       [1.00137533, 0.05539176, 1.74832   ]])

In [7]:
toy_x = np.random.randn(10,3)
toy_mu = np.mean(toy_x, 0)
print(toy_x.shape, toy_mu.shape)

(10, 3) (3,)


In [8]:
np.cov(toy_x, rowvar=0)

array([[ 0.82739692, -0.04873264,  0.09037384],
       [-0.04873264,  0.41972606,  0.06164041],
       [ 0.09037384,  0.06164041,  1.76451134]])

In [9]:
from numpy.core.fromnumeric import argmax
from math import pi
from numpy.random import Philox
class GDA:
  def __init__(self):
    ## set mu, phi and sigma to None
    self.mu = None
    self.phi =None
    self.sigma = None

  def fit(self,x,y):
    k=np.unique(y).shape[0] # Number of class.
    d=x.shape[1]  # input dim
    m=x.shape[0]  # Number of examples.
    
    ## Initialize mu, phi and sigma
    self.mu= np.zeros((k,d))#: kxd, i.e., each row contains an individual class mu.
    self.sigma= np.zeros((k,d,d))#: kxdxd, i.e., each row contains an individual class sigma.
    self.phi= np.zeros(k)# d-dimension
    for label in range(k):
      self.phi[label] = np.sum(label == y)/x.shape[0]
      #idx=np.where(y==label)
      self.mu[label] = np.mean(x[label == y], axis=0)
      self.sigma[label] = covariance(x[label == y], self.mu[label])

    ## START THE LEARNING: estimate mu, phi and sigma.
    




  def predict_proba(self,x):
    # reshape or flatt x.
    #x= x.reshape[0]
    d=x.shape[1]
    k_class= self.mu.shape[0]# Number of classes we have in our case it's k = 2
    print(x.shape[0])
    ## START THE LEARNING: estimate mu, phi and sigma.
    #mu = self.mu
    #phi =self.phi
    #sigma = self.sigma
    pro = np.zeros((x.shape[0], k_class))
    
    for lab in range(k_class):
      for i in range(x.shape[0]):
        coeff = 1/((2*pi)**(d/2)) *(np.linalg.det(self.sigma[lab])**(1/2))
        pro[i,lab] = coeff * np.exp(-(1/2)*(x[i] - self.mu[lab]).T@np.linalg.inv(self.sigma[lab])@(x[i] - self.mu[lab]))
    return pro



  def predict(self,x):
    predict_proba = self.predict_proba(x)*self.phi
    y_pred = np.argmax(predict_proba, axis = 1)
    return y_pred
  
  def accuracy(self, y, ypreds):
    accu = np.mean(y==ypreds)
    return accu

In [10]:
model= GDA()
model.fit(x_train,y_train)

In [11]:
model.mu

array([[ 1.00201779,  1.00494482,  0.99548846],
       [-1.01641736,  0.96824826, -0.90433708]])

In [12]:
model.phi

array([0.495, 0.505])

In [13]:
model.sigma

array([[[ 0.90164773, -0.43335029, -0.06500866],
        [-0.43335029,  1.68747351,  0.10027055],
        [-0.06500866,  0.10027055,  0.0384381 ]],

       [[ 0.7647548 ,  0.35278165,  0.0911074 ],
        [ 0.35278165,  0.34171233, -0.08598807],
        [ 0.0911074 , -0.08598807,  1.62087217]]])

In [14]:
yproba = model.predict_proba(x_test)
yproba.shape

200


(200, 2)

In [15]:
ypreds= model.predict(x_test)
ypreds


200


array([0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0,
       1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0,
       1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1,
       0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1,
       0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1,
       0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1,
       1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1,
       1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1,
       1, 0])

In [16]:
model.accuracy(y_test, ypreds)

0.965

In [21]:
from sklearn.utils.validation import check_X_y
class LogisticRegression:           
  '''
  The goal of this class is to create a LogisticRegression class, 
  that we will use as our model to classify data point into a corresponding class
  '''
  def __init__(self,lr,n_epochs):
    self.lr = lr
    self.n_epochs = n_epochs
    self.train_losses = []
    self.w = None
    self.weight = []

  def add_ones(self, x):
    ##### WRITE YOUR CODE HERE #####
    N = np.ones((x.shape[0],1))
    return np.hstack((N,x))
  
    #### END CODE ####

  def sigmoid(self, x):
    ##### WRITE YOUR CODE HERE ####
    sig = 1/(1 + np.exp(-x@self.w))
    return sig
    #### END CODE ####
    

  def cross_entropy(self, x, y_true):
    ##### WRITE YOUR CODE HERE #####
    y_pred = self.sigmoid(x)
    N,D = x.shape
    loss = (-1/N)* np.sum(y_true * np.log(y_pred) + (1- y_true) * np.log(1 - y_pred))
    return loss
    #### END CODE ####
  
  def predict_proba(self,x):  #This function will use the sigmoid function to compute the probalities
    ##### WRITE YOUR CODE HERE #####
    x= self.add_ones(x)
    proba = self.sigmoid(x)
    return proba
    #### END CODE ####

  def predict(self,x):
    ##### WRITE YOUR CODE HERE #####
    probas = self.predict_proba(x)
    output = [0 if p<0.5 else 1 for p in probas]#np.where(probas > 0.5, 1, 0)      #convert the probalities into 0 and 1 by using a treshold=0.5
    return output
    #### END CODE ####

  def fit(self,x,y):
    # Add ones to x
    x = self.add_ones(x)

    # reshape y if needed
    y = y.reshape(-1,1)

    # Initialize w to zeros vector >>> (x.shape[1])
    self.w = np.zeros((x.shape[1],1))

    for epoch in range(self.n_epochs):
      # make predictions
      ypred = self.sigmoid(x)

      #compute the gradient

      dl = (-1/x.shape[0])*x.T@(y- ypred)

      #update rule
      self.w = self.w - self.lr * dl

      #Compute and append the training loss in a list
      loss = self.cross_entropy(x, y)

      if epoch%100 == 0:
        print(f'loss for epoch {epoch}  : {loss}')

  def accuracy(self,y_true, y_pred):
    ##### WRITE YOUR CODE HERE #####
    acc = np.mean(y_true==y_pred)*100
    return acc
    #### END CODE ####

In [23]:
model= LogisticRegression(lr=0.01,n_epochs=10000)
model.fit(x_train,y_train)

loss for epoch 0  : 0.688363021140951
loss for epoch 100  : 0.422640627331724
loss for epoch 200  : 0.32872318242874854
loss for epoch 300  : 0.2821421669089025
loss for epoch 400  : 0.2542740474046796
loss for epoch 500  : 0.23564711029252902
loss for epoch 600  : 0.22226529615438467
loss for epoch 700  : 0.21215414670829633
loss for epoch 800  : 0.20422490633561083
loss for epoch 900  : 0.1978269301185786
loss for epoch 1000  : 0.1925467797763679
loss for epoch 1100  : 0.1881087412082297
loss for epoch 1200  : 0.1843216354180446
loss for epoch 1300  : 0.18104857389456974
loss for epoch 1400  : 0.17818886930242633
loss for epoch 1500  : 0.17566674984854355
loss for epoch 1600  : 0.1734240619215231
loss for epoch 1700  : 0.17141540446185388
loss for epoch 1800  : 0.16960479715861454
loss for epoch 1900  : 0.16796334511586686
loss for epoch 2000  : 0.1664675679074136
loss for epoch 2100  : 0.1650981819047337
loss for epoch 2200  : 0.16383919824898385
loss for epoch 2300  : 0.16267724470

In [24]:
yproba = model.predict_proba(x_test)
yproba.shape

(200, 1)

In [26]:
model.accuracy(y_train,y_train)

100.0